# DCR Extension Tutorial

This file will give a walkthrough of the extension that has been made for the PM4Py library.
Namely, it will go through how to create a DCR Graph,  either manually or automatically by the implemented DisCoveR algorithm, showcase how conformance checking can be used to determine fitness, and finally present the import/export capability such that the graph can be visualised.


## Creating DCR Graphs
First, let's take a look at how to create a basic DCR Graph manually. Note that the underlying attribute in the DCR graph objects consists of sets and dictionaries, so one can access them through the property, and call them as such:

In [1]:
from pm4py.objects.dcr.obj import DcrGraph
graph = DcrGraph()
graph.events.add("A")
graph.events.add("B")
graph.events.add("C")
graph.labels.add("activity1")
graph.labels.add("activity2")
graph.labels.add("activity3")
graph.label_mapping["activity1"] = {"A"}
graph.label_mapping["activity2"] = {"B"}
graph.label_mapping["activity3"] = {"C"}
graph.conditions["A"] = set("B")
graph.conditions["B"] = set("C")
graph.responses["A"] = set("C")
graph.excludes["C"] = set("B")
graph.conditions["A"].add("D")
graph.includes["A"] = set("C")
graph.marking.included.add("A")
graph.marking.included.add("B")
graph.marking.included.add("C")
graph.marking.included.add("D")
print(graph)

events: {'C', 'B', 'A'}
marking: {executed: set(), included: {'C', 'B', 'A', 'D'}, pending: set()}
labels: {'activity2', 'activity1', 'activity3'}
conditionsFor: {'A': {'B', 'D'}, 'B': {'C'}}
responseTo: {'A': {'C'}}
includesTo: {'A': {'C'}}
excludesTo: {'C': {'B'}}
labelMapping: {'activity1': {'A'}, 'activity2': {'B'}, 'activity3': {'C'}}


C:\Users\herma\pm4py-dcr\pm4py\objects\dcr\obj.py:251: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if event is "None":


When the graph has been constructued, one has access to the following commands to get the different properties of the DCR graph, such as the size defined by the number of constraints, the events associated with the activity or vice versa.

In [2]:
print(graph.get_constraints())
print(graph.get_activity("A"))
print(graph.get_event("activity1"))
del graph

6
activity1
A


If one calls get_event() or get_activity() with a value that doesn't exists, it will return the input value. This was implemted for the conformance checking, since if someones tries to get the labelmapping but the the event doesn't exist, the test would be interrupted. This way, if it doesn't exist, it will be noted by the conformance tools and be used in the conformance result.

Now to discover a model with a given input log, a simplified interface has been created, such that the implemented algorithms are easily accessible, and therefore allow for a more simplified and straightforward use.

In [3]:
import pm4py

log = pm4py.read_xes("../tests/input_data/running-example.xes")
graph, _ = pm4py.discover_dcr(log)
print(graph)
del graph

parsing log, completed traces ::   0%|          | 0/6 [00:00<?, ?it/s]

events: {'decide', 'reinitiate request', 'reject request', 'examine casually', 'pay compensation', 'register request', 'examine thoroughly', 'check ticket'}
marking: {executed: set(), included: {'decide', 'reinitiate request', 'reject request', 'examine casually', 'pay compensation', 'register request', 'examine thoroughly', 'check ticket'}, pending: set()}
labels: {'decide', 'reinitiate request', 'examine casually', 'pay compensation', 'register request', 'examine thoroughly', 'check ticket', 'reject request'}
conditionsFor: {'reinitiate request': {'decide'}, 'pay compensation': {'decide'}, 'reject request': {'decide'}, 'decide': {'examine casually', 'check ticket'}, 'examine casually': {'register request'}, 'examine thoroughly': {'register request'}, 'check ticket': {'register request'}}
responseTo: {'reinitiate request': {'check ticket'}, 'register request': {'check ticket'}, 'examine casually': {'decide'}, 'examine thoroughly': {'decide'}, 'check ticket': {'decide'}}
includesTo: {'

In accordance to the rest of the library, the simplified interface takes in extra values for which one can use to specify the naming convention in the attribute, such that it can mine the log without failure.

Additionally, the discover miner has been extended to allow for mining of roles; we will use the log of the running example once again.

In [4]:
graph, _ = pm4py.discover_dcr(log,process_type={"roles"},group_key="org:resource")
print(graph)
del graph

events: {'decide', 'reinitiate request', 'reject request', 'examine casually', 'pay compensation', 'register request', 'examine thoroughly', 'check ticket'}
marking: {executed: set(), included: {'decide', 'reinitiate request', 'reject request', 'examine casually', 'pay compensation', 'register request', 'examine thoroughly', 'check ticket'}, pending: set()}
labels: {'decide', 'reinitiate request', 'examine casually', 'pay compensation', 'register request', 'examine thoroughly', 'check ticket', 'reject request'}
conditionsFor: {'reinitiate request': {'decide'}, 'pay compensation': {'decide'}, 'reject request': {'decide'}, 'decide': {'examine casually', 'check ticket'}, 'examine casually': {'register request'}, 'examine thoroughly': {'register request'}, 'check ticket': {'register request'}}
responseTo: {'reinitiate request': {'check ticket'}, 'register request': {'check ticket'}, 'examine casually': {'decide'}, 'examine thoroughly': {'decide'}, 'check ticket': {'decide'}}
includesTo: {'

If one doesn't wish to use the simplified interface, there is a more indirect way of performing the process mining technique

In [5]:
from pm4py.algo.discovery.dcr_discover.variants.dcr_discover import Discover
from pm4py.objects.dcr.obj import DcrGraph
disc = Discover()
graph = DcrGraph(disc.mine(log)[0])
del log

Note that the discover miner also returns the abstraction log used for mining the DCR Graph, therefore it is needed to perform to specify the first iterative of the tuple.

## Conformance Checking
Two different techniques for checking conformance of a DCR graph have been implemented
### Rule Checking
The first technique is a quite straight forward approach. It takes an event log, and mines based on the constraints within the graph. This technique takes in a whole log and produces an output: a list of dictionaries of the values associated with the mining:

In [6]:
log = pm4py.read_xes("../tests/input_data/running-example.xes")
graph, _ = pm4py.discover_dcr(log)
conf_res = pm4py.conformance_dcr(log, graph)
print(conf_res)
del conf_res
del graph
del log

parsing log, completed traces ::   0%|          | 0/6 [00:00<?, ?it/s]

[{'no_constr_total': 30, 'deviations': [], 'no_dev_total': 0, 'dev_fitness': 1.0, 'is_fit': True}, {'no_constr_total': 30, 'deviations': [], 'no_dev_total': 0, 'dev_fitness': 1.0, 'is_fit': True}, {'no_constr_total': 30, 'deviations': [], 'no_dev_total': 0, 'dev_fitness': 1.0, 'is_fit': True}, {'no_constr_total': 30, 'deviations': [], 'no_dev_total': 0, 'dev_fitness': 1.0, 'is_fit': True}, {'no_constr_total': 30, 'deviations': [], 'no_dev_total': 0, 'dev_fitness': 1.0, 'is_fit': True}, {'no_constr_total': 30, 'deviations': [], 'no_dev_total': 0, 'dev_fitness': 1.0, 'is_fit': True}]


Due to the DisCoveR miner's property of always producing a graph with perfect fitness, the output will therefore have no deviations. In addition to mining of DCR graphs, it is also possible to check for deviations in role assignment.

In [7]:
# Given an event log and discovering a dcr
log = pm4py.read_xes("../tests/input_data/running-example.xes")
graph, _ = pm4py.discover_dcr(log, process_type={'roles'}, group_key="org:resource")

# when the roles are changed and conformance is performed
log = log.replace("Mike", "Brenda")
conf_res = pm4py.conformance_dcr(log, graph, group_key="org:resource")
print(conf_res)

parsing log, completed traces ::   0%|          | 0/6 [00:00<?, ?it/s]

[{'no_constr_total': 49, 'deviations': [('roleViolation', 'Brenda')], 'no_dev_total': 1, 'dev_fitness': 0.9795918367346939, 'is_fit': False}, {'no_constr_total': 49, 'deviations': [('roleViolation', 'Brenda')], 'no_dev_total': 1, 'dev_fitness': 0.9795918367346939, 'is_fit': False}, {'no_constr_total': 49, 'deviations': [('roleViolation', 'Brenda')], 'no_dev_total': 1, 'dev_fitness': 0.9795918367346939, 'is_fit': False}, {'no_constr_total': 49, 'deviations': [('roleViolation', 'Brenda')], 'no_dev_total': 1, 'dev_fitness': 0.9795918367346939, 'is_fit': False}, {'no_constr_total': 49, 'deviations': [('roleViolation', 'Brenda')], 'no_dev_total': 1, 'dev_fitness': 0.9795918367346939, 'is_fit': False}, {'no_constr_total': 49, 'deviations': [('roleViolation', 'Brenda')], 'no_dev_total': 1, 'dev_fitness': 0.9795918367346939, 'is_fit': False}]


If one wishes, one can return the values as a pandas DataFrame:


In [8]:
log = pm4py.read_xes("../tests/input_data/running-example.xes")
graph, _ = pm4py.discover_dcr(log, process_type={'roles'}, group_key="org:resource")

# when the roles are changed and conformance is performed
log = log.replace("Mike", "Brenda")
conf_res = pm4py.conformance_dcr(log, graph, group_key="org:resource",return_diagnostics_dataframe=True)
print(conf_res)



parsing log, completed traces ::   0%|          | 0/6 [00:00<?, ?it/s]

  case_id  no_dev_total  no_constr_total  dev_fitness
0       3             1               49     0.979592
1       2             1               49     0.979592
2       1             1               49     0.979592
3       6             1               49     0.979592
4       5             1               49     0.979592
5       4             1               49     0.979592


C:\Users\herma\pm4py-dcr\pm4py\utils.py:542: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")


Just like in the previous example, one can call the under lying algorithm directly if they wish to do so. In this case, one can call the class used for conformance checking:

In [9]:
from pm4py.algo.conformance.dcr.variants.classic import RuleBasedConformance
parameters = pm4py.utils.get_properties(log,group_key="org:resource")
rulecheck = RuleBasedConformance(log, graph,parameters)
conf_res = rulecheck.apply_conformance()
print(conf_res)

[{'no_constr_total': 49, 'deviations': [('roleViolation', 'Brenda')], 'no_dev_total': 1, 'dev_fitness': 0.9795918367346939, 'is_fit': False}, {'no_constr_total': 49, 'deviations': [('roleViolation', 'Brenda')], 'no_dev_total': 1, 'dev_fitness': 0.9795918367346939, 'is_fit': False}, {'no_constr_total': 49, 'deviations': [('roleViolation', 'Brenda')], 'no_dev_total': 1, 'dev_fitness': 0.9795918367346939, 'is_fit': False}, {'no_constr_total': 49, 'deviations': [('roleViolation', 'Brenda')], 'no_dev_total': 1, 'dev_fitness': 0.9795918367346939, 'is_fit': False}, {'no_constr_total': 49, 'deviations': [('roleViolation', 'Brenda')], 'no_dev_total': 1, 'dev_fitness': 0.9795918367346939, 'is_fit': False}, {'no_constr_total': 49, 'deviations': [('roleViolation', 'Brenda')], 'no_dev_total': 1, 'dev_fitness': 0.9795918367346939, 'is_fit': False}]


### Alignment

The extension also allows for determining the optimal alignment of a DCR Graph.

In [10]:
log = pm4py.read_xes("../tests/input_data/running-example.xes")
graph, _ = pm4py.discover_dcr(log)
align_res = pm4py.optimal_alignment_dcr(log, graph)
align_res

parsing log, completed traces ::   0%|          | 0/6 [00:00<?, ?it/s]

[{'alignment': [('register request', 'register request'),
   ('examine thoroughly', 'examine thoroughly'),
   ('check ticket', 'check ticket'),
   ('decide', 'decide'),
   ('reject request', 'reject request')],
  'cost': 0,
  'visited_states': 7,
  'closed': 6,
  'global_min': 0,
  'fitness': 1.0,
  'bwc': 5},
 {'alignment': [('register request', 'register request'),
   ('check ticket', 'check ticket'),
   ('examine casually', 'examine casually'),
   ('decide', 'decide'),
   ('pay compensation', 'pay compensation')],
  'cost': 0,
  'visited_states': 8,
  'closed': 6,
  'global_min': 0,
  'fitness': 1.0,
  'bwc': 5},
 {'alignment': [('register request', 'register request'),
   ('examine casually', 'examine casually'),
   ('check ticket', 'check ticket'),
   ('decide', 'decide'),
   ('reinitiate request', 'reinitiate request'),
   ('examine thoroughly', 'examine thoroughly'),
   ('check ticket', 'check ticket'),
   ('decide', 'decide'),
   ('pay compensation', 'pay compensation')],
  'co

Once again, this can be viewed as a DataFrame:

In [11]:
align_res = pm4py.optimal_alignment_dcr(log, graph, return_diagnostics_dataframe=True)
align_res

C:\Users\herma\pm4py-dcr\pm4py\utils.py:542: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")


,case_id,align_fitness
0,3,1.0
1,2,1.0
2,1,1.0
3,6,1.0
4,5,1.0
5,4,1.0


## Importing/Exporting DCR Graphs
We can visualise the mined DCR graph by exporting it as an 'xml' file and then viewing it at the corresponding portal.

In [12]:
from pm4py.objects.dcr.exporter import exporter as dcr_exporter
log = pm4py.read_xes("../tests/input_data/running-example.xes")
graph, _ = pm4py.discover_dcr(log)
path = '../tests/test_output_data/dcrgraph.xml'
pm4py.write_dcr_xml(graph, path, variant=dcr_exporter.DCR_JS_PORTAL, dcr_title='dcrgraph')

parsing log, completed traces ::   0%|          | 0/6 [00:00<?, ?it/s]

In this case, we can visualise the graph at the following link: https://hugoalopez-dtu.github.io/dcr-js/.

Importing the graph is just as easy.

In [13]:
from pm4py.objects.dcr.importer import importer as dcr_importer
graph = pm4py.read_dcr_xml(path, variant=dcr_importer.DCR_JS_PORTAL)
graph

events: {'decide', 'reinitiate request', 'pay compensation', 'examine casually', 'register request', 'examine thoroughly', 'check ticket', 'reject request'}
marking: {executed: set(), included: {'decide', 'reinitiate request', 'pay compensation', 'examine casually', 'register request', 'examine thoroughly', 'check ticket', 'reject request'}, pending: set()}
labels: {'decide', 'reinitiate request', 'pay compensation', 'examine casually', 'register request', 'examine thoroughly', 'check ticket', 'reject request'}
conditionsFor: {'pay compensation': {'decide'}, 'reject request': {'decide'}, 'reinitiate request': {'decide'}, 'examine thoroughly': {'register request'}, 'check ticket': {'register request'}, 'examine casually': {'register request'}, 'decide': {'examine casually', 'check ticket'}}
responseTo: {'register request': {'check ticket'}, 'examine thoroughly': {'decide'}, 'check ticket': {'decide'}, 'examine casually': {'decide'}, 'reinitiate request': {'check ticket'}}
includesTo: {'

One can alter the graph using the DCR-js tool and then import it back into PM4Py for further manipulation, for instance checking the alignment of the new model.

In [16]:
path_edited = '../tests/test_output_data/dcrgraph_edited.xml'
graph_edited = pm4py.read_dcr_xml(path_edited, variant=dcr_importer.DCR_JS_PORTAL)
align_res = pm4py.optimal_alignment_dcr(log, graph_edited, return_diagnostics_dataframe=True)
align_res

C:\Users\herma\pm4py-dcr\pm4py\utils.py:542: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")


,case_id,align_fitness
0,3,0.833333
1,2,0.800000
2,1,0.600000
3,6,0.800000
4,5,0.923077
5,4,0.600000


In [17]:
# cleanup
import os
os.remove(path)
os.remove(path_edited)